In [16]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from datetime import datetime
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType
import pandas as pd

In [12]:
def merge_data(spark: SparkSession, output_dir: str, start_date: str, end_date: str):
    sy, sm = start_date.split('-')
    ey, em = end_date.split('-')
    tmp_df = []
    for year in range(int(sy), int(ey) + 1):
        for month in range(1, 13):
            if year == int(sy) and month < int(sm):
                continue
            if year == int(ey) and month > int(em):
                continue
            file_path = f'{output_dir}/fhvhv_tripdata_{year}-{str(month).zfill(2)}.parquet'
            tmp_df.append(spark.read.parquet(file_path))

    if len(tmp_df) == 1:
        return tmp_df[0]
    else:
        tmp = tmp_df[0]
        for idx in range(1, len(tmp_df)):
            tmp = tmp.union(tmp_df[idx])
        return tmp

In [13]:
def clean_data(row):
    """Remove missing/invalid data"""
    try:
        pickup_datetime, trip_mile, base_passenger_fare = row
        if not all([
            pickup_datetime is not None,
            trip_mile is not None,
            base_passenger_fare is not None
        ]):
            return False

        if trip_mile <= 0. or base_passenger_fare <= 0.:
            return False
        return True
    except Exception as e:
        print(f"Error in row validation: {e}")
        return False

In [15]:
spark = SparkSession.builder \
    .appName("NYC Taxi Data Analysis") \
    .master("spark://spark-master:7077") \
    .config("spark.executor.memory", "20g") \
    .getOrCreate()

/opt/homebrew/Cellar/apache-spark/3.5.1/bin/load-spark-env.sh: line 2: /opt/homebrew/Cellar/apache-spark/3.5.1/libexec/bin/load-spark-env.sh: Permission denied
/opt/homebrew/Cellar/apache-spark/3.5.1/bin/load-spark-env.sh: line 2: exec: /opt/homebrew/Cellar/apache-spark/3.5.1/libexec/bin/load-spark-env.sh: cannot execute: Undefined error: 0


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.